# Preparación de Datos
En esta sección limpiamos y transformamos el dataset para dejarlo listo para el análisis exploratorio y los modelos:
- Cargar datos con columnas clave y tipos adecuados.
- Revisar y tratar valores faltantes.
- Convertir tipos (fechas, booleanos) y crear variables derivadas.
- Guardar un dataset limpio para uso posterior.

In [1]:
# Librerías y rutas
import os
from pathlib import Path
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)
data_path = Path("US_Accidents_March23.csv")
if not data_path.exists():
    raise FileNotFoundError(f"No se encontró el archivo en {data_path.resolve()}")

### Selección de columnas y parseo de fechas
- **Subconjunto de columnas**: severidad, tiempos (inicio/fin), coordenadas inicio/fin, distancia, contexto vial (calle, ciudad, estado), clima detallado y marcas de iluminación/amanecer. Esto reduce memoria y centra el análisis en factores plausibles.
- **`parse_dates`**: convierte `Start_Time`, `End_Time` y `Weather_Timestamp` a `datetime` al leer, evitando conversiones posteriores y preservando precisión temporal.

In [2]:
cols_to_use = [
    "ID", "Source", "Severity",
    "Start_Time", "End_Time", "Start_Lat", "Start_Lng", "End_Lat", "End_Lng", "Distance(mi)",
    "Description", "Street", "City", "County", "State", "Zipcode", "Country", "Timezone", "Airport_Code",
    "Weather_Timestamp", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)",
    "Wind_Direction", "Wind_Speed(mph)", "Precipitation(in)", "Weather_Condition",
    "Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit", "Railway",
    "Roundabout", "Station", "Stop", "Traffic_Calming", "Traffic_Signal", "Turning_Loop",
    "Sunrise_Sunset", "Civil_Twilight", "Nautical_Twilight", "Astronomical_Twilight"
]

raw_df = pd.read_csv(
    data_path,
    usecols=cols_to_use,
    parse_dates=["Start_Time", "End_Time", "Weather_Timestamp"],
    low_memory=False
)

raw_df.head(), raw_df.shape

(    ID   Source  Severity           Start_Time             End_Time  \
 0  A-1  Source2         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
 1  A-2  Source2         2  2016-02-08 06:07:59  2016-02-08 06:37:59   
 2  A-3  Source2         2  2016-02-08 06:49:27  2016-02-08 07:19:27   
 3  A-4  Source2         3  2016-02-08 07:23:34  2016-02-08 07:53:34   
 4  A-5  Source2         2  2016-02-08 07:39:07  2016-02-08 08:09:07   
 
    Start_Lat  Start_Lng  End_Lat  End_Lng  Distance(mi)  \
 0  39.865147 -84.058723      NaN      NaN          0.01   
 1  39.928059 -82.831184      NaN      NaN          0.01   
 2  39.063148 -84.032608      NaN      NaN          0.01   
 3  39.747753 -84.205582      NaN      NaN          0.01   
 4  39.627781 -84.188354      NaN      NaN          0.01   
 
                                          Description  \
 0  Right lane blocked due to accident on I-70 Eas...   
 1  Accident on Brice Rd at Tussing Rd. Expect del...   
 2  Accident on OH-32 State Route

### Revisión de valores faltantes
- Conocer el % de faltantes guía si se imputan, se eliminan columnas o se ajusta el umbral.
- Ordenar de mayor a menor faltante permite focalizar la limpieza en las variables más problemáticas.


In [3]:
missing_pct = raw_df.isna().mean().sort_values(ascending=False)
missing_pct

End_Lng                  4.402935e-01
End_Lat                  4.402935e-01
Precipitation(in)        2.851286e-01
Wind_Chill(F)            2.586590e-01
Wind_Speed(mph)          7.391355e-02
Visibility(mi)           2.291524e-02
Wind_Direction           2.267043e-02
Humidity(%)              2.253301e-02
Weather_Condition        2.244438e-02
Temperature(F)           2.120143e-02
Pressure(in)             1.820288e-02
Weather_Timestamp        1.555666e-02
Sunrise_Sunset           3.007869e-03
Civil_Twilight           3.007869e-03
Astronomical_Twilight    3.007869e-03
Nautical_Twilight        3.007869e-03
Airport_Code             2.928810e-03
Street                   1.406372e-03
Timezone                 1.010300e-03
Zipcode                  2.477876e-04
City                     3.273643e-05
Description              6.469649e-07
ID                       0.000000e+00
Distance(mi)             0.000000e+00
Start_Lng                0.000000e+00
Source                   0.000000e+00
Severity    

### Esquema de limpieza e imputación
- Umbral >40% faltantes: columnas muy incompletas añaden ruido; si lo superan, se eliminan para conservar calidad.
- Booleanos a 0/1: estandariza amenidades/señalización (incluye `Turning_Loop`) para modelos y agrupaciones.
- Imputación: mediana para numéricos (robusta a outliers) y moda para categóricas; si no hay moda, se marca "Desconocido".
- Filtrado: sin lat/lon, severidad o tiempos (inicio/fin) la observación no sirve para geolocalizar, calcular duración ni para el objetivo.

In [4]:
# Eliminar accidentes sin fecha de inicio o fin
clean_df = raw_df.copy()
del raw_df  # Liberar memoria del DataFrame original
clean_df.dropna(subset=["Start_Time", "End_Time"], inplace=True)

# Eliminar columnas con >40% faltantes (si alguna superó el umbral)
threshold = 0.40
missing_pct = clean_df.isna().mean().sort_values(ascending=False)
high_missing_cols = missing_pct[missing_pct > threshold].index.tolist()
if high_missing_cols:
    clean_df = clean_df.drop(columns=high_missing_cols)
    print(f"Columnas eliminadas por alto faltante (>40%): {high_missing_cols}")
else:
    print("No se eliminaron columnas por faltantes altos.")

# Asegurar tipos: columnas booleanas en 0/1
bool_cols = [c for c in clean_df.columns if c in [
    "Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit",
    "Railway", "Roundabout", "Station", "Stop", "Traffic_Calming", "Traffic_Signal", "Turning_Loop"
]]
for c in bool_cols:
    clean_df[c] = clean_df[c].fillna(False).astype(int)

# Imputación simple
num_cols = clean_df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = clean_df.select_dtypes(include=["object"]).columns.tolist()

for c in num_cols:
    clean_df[c] = clean_df[c].fillna(clean_df[c].median())

# Excluir columnas datetime de la imputación categórica
date_cols = ["Start_Time", "End_Time", "Weather_Timestamp"]
cat_cols = [c for c in cat_cols if c not in date_cols]

for c in cat_cols:
    mode_val = clean_df[c].mode(dropna=True)
    if not mode_val.empty:
        clean_df[c] = clean_df[c].fillna(mode_val.iloc[0])
    else:
        clean_df[c] = clean_df[c].fillna("Desconocido")

# Reconvertir columnas datetime (por si perdieron el tipo durante las operaciones)
for date_col in date_cols:
    if date_col in clean_df.columns:
        clean_df[date_col] = pd.to_datetime(clean_df[date_col], errors='coerce')

# Eliminar filas sin coordenadas, severidad o fechas (una sola operación)
required_cols = ["Start_Lat", "Start_Lng", "Severity", "Start_Time", "End_Time"]
clean_df = clean_df.dropna(subset=required_cols)

clean_df.head(), clean_df.shape

Columnas eliminadas por alto faltante (>40%): ['End_Lng', 'End_Lat']


(    ID   Source  Severity          Start_Time            End_Time  Start_Lat  \
 0  A-1  Source2         3 2016-02-08 05:46:00 2016-02-08 11:00:00  39.865147   
 1  A-2  Source2         2 2016-02-08 06:07:59 2016-02-08 06:37:59  39.928059   
 2  A-3  Source2         2 2016-02-08 06:49:27 2016-02-08 07:19:27  39.063148   
 3  A-4  Source2         3 2016-02-08 07:23:34 2016-02-08 07:53:34  39.747753   
 4  A-5  Source2         2 2016-02-08 07:39:07 2016-02-08 08:09:07  39.627781   
 
    Start_Lng  Distance(mi)                                        Description  \
 0 -84.058723          0.01  Right lane blocked due to accident on I-70 Eas...   
 1 -82.831184          0.01  Accident on Brice Rd at Tussing Rd. Expect del...   
 2 -84.032608          0.01  Accident on OH-32 State Route 32 Westbound at ...   
 3 -84.205582          0.01  Accident on I-75 Southbound at Exits 52 52B US...   
 4 -84.188354          0.01  Accident on McEwen Rd at OH-725 Miamisburg Cen...   
 
                  

### Variables derivadas y filtrado de anomalías
- Duración captura severidad temporal y exposición; hora y día capturan patrones de movilidad y congestión.
- Se descartan duraciones negativas (errores) o mayores a ~7 días (~10,000 min) como registros anómalos que distorsionarían estadísticos y modelos.

In [5]:
# Duración del accidente en minutos
clean_df["duration_min"] = (clean_df["End_Time"] - clean_df["Start_Time"]).dt.total_seconds() / 60

# Eliminar duraciones inválidas inmediatamente (antes de crear más columnas)
# Identificar filas inválidas y eliminarlas con drop
invalid_idx = clean_df[(clean_df["duration_min"] < 0) | (clean_df["duration_min"] > 1e4)].index
clean_df.drop(invalid_idx, inplace=True)
del invalid_idx
clean_df.reset_index(drop=True, inplace=True)

# Hora y día de la semana
clean_df["start_hour"] = clean_df["Start_Time"].dt.hour
clean_df["start_dow"] = clean_df["Start_Time"].dt.dayofweek  # 0=lunes

clean_df[["duration_min", "start_hour", "start_dow"]].describe(include="all")

,duration_min,start_hour,start_dow
count,6.979296e+06,6.979296e+06,6.979296e+06
mean,1.126926e+02,1.226775e+01,2.568613e+00
std,1.879010e+02,5.454558e+00,1.796749e+00
min,1.216667e+00,0.000000e+00,0.000000e+00
25%,3.000000e+01,8.000000e+00,1.000000e+00
50%,6.258333e+01,1.300000e+01,3.000000e+00
75%,1.225333e+02,1.700000e+01,4.000000e+00
max,9.993067e+03,2.300000e+01,6.000000e+00


### Features adicionales para modelado
- Extraer variables temporales (hora, día de la semana, mes, año)
- Crear categorías de momento del día para capturar patrones de movilidad

In [6]:
# Extraer mes y año
clean_df["start_month"] = clean_df["Start_Time"].dt.month
clean_df["start_year"] = clean_df["Start_Time"].dt.year

# Crear categoría de momento del día
def categorize_time(hour):
    if 6 <= hour < 12:
        return 'Mañana'
    elif 12 <= hour < 18:
        return 'Tarde'
    elif 18 <= hour < 22:
        return 'Noche'
    else:
        return 'Madrugada'

clean_df["time_category"] = clean_df["start_hour"].apply(categorize_time)

print("✓ Features temporales adicionales creadas")
print(f"Rango de años: {clean_df['start_year'].min()} - {clean_df['start_year'].max()}")
print(f"\nDistribución por momento del día:")
print(clean_df["time_category"].value_counts())

✓ Features temporales adicionales creadas
Rango de años: 2016 - 2023

Distribución por momento del día:
time_category
Tarde        2562119
Mañana       2432072
Noche        1028053
Madrugada     957052
Name: count, dtype: int64


### Guardar dataset limpio
- Dataset completo con todas las transformaciones listas para análisis exploratorio y modelado

In [7]:
# Guardar dataset limpio
output_csv = Path("accidents_clean.csv")

clean_df.to_csv(output_csv, index=False)
print(f"Dataset limpio guardado en: {output_csv.resolve()}")
clean_df.head()

Dataset limpio guardado en: C:\Traffic_Accidents_Stats_Analysis\accidents_clean.csv


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,duration_min,start_hour,start_dow,start_month,start_year,time_category
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,62.0,91.0,29.68,10.0,Calm,7.0,0.02,Light Rain,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Night,314.0,5,0,2,2016,Madrugada
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,62.0,100.0,29.65,10.0,Calm,7.0,0.00,Light Rain,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Night,Night,Day,30.0,6,0,2,2016,Mañana
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,0.00,Overcast,0,0,0,0,0,0,0,0,0,0,0,1,0,Night,Night,Day,Day,30.0,6,0,2,2016,Mañana
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,0,0,Night,Day,Day,Day,30.0,7,0,2,2016,Mañana
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,0.00,Mostly Cloudy,0,0,0,0,0,0,0,0,0,0,0,1,0,Day,Day,Day,Day,30.0,7,0,2,2016,Mañana
